In [0]:
%run "../includes/common_code"

Using race_results from presentation layer to get constructor standings

In [0]:
race_results_df = spark.read.parquet(f"{presentation_container_path}/race_results")

In [0]:
from pyspark.sql.functions import col, when, count, sum, desc, asc

First way to get constructor standings but this don't has rank column but shows the correct order

In [0]:
constructor_standings_1 = race_results_df \
    .groupBy(race_results_df.race_year, race_results_df.team_name) \
        .agg(sum(col('points')).alias('points'), count(when(col('position') == 1, True)).alias('Wins')) \
            .orderBy(desc(col('points')), desc(col('Wins')))

In [0]:
display(constructor_standings_1.filter(constructor_standings_1.race_year == 2020))

race_year,team_name,points,Wins
2020,Mercedes,573.0,13
2020,Red Bull,319.0,2
2020,Racing Point,210.0,1
2020,McLaren,202.0,0
2020,Renault,181.0,0
2020,Ferrari,131.0,0
2020,AlphaTauri,107.0,1
2020,Alfa Romeo,8.0,0
2020,Haas F1 Team,3.0,0
2020,Williams,0.0,0


Lets do it using window functions, with this you can get rank

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

In [0]:
constructor_standings_2 = race_results_df \
    .groupBy(race_results_df.race_year, race_results_df.team_name) \
        .agg(sum(col('points')).alias('points'), count(when(col('position') == 1, True)).alias('Wins'))

In [0]:
constructor_rank = Window.partitionBy('race_year').orderBy(desc(col('points')), desc(col('Wins')))
constructor_standings_2 = constructor_standings_2.withColumn('rank', rank().over(constructor_rank))

In [0]:
# Lets reorder according to the BBC driver standings page
constructor_standings_final = constructor_standings_2.select(constructor_standings_2.rank, constructor_standings_2.team_name, constructor_standings_2.Wins, constructor_standings_2.points, constructor_standings_2.race_year)

In [0]:
display(constructor_standings_final.filter(constructor_standings_final.race_year == 2020))

rank,team_name,Wins,points,race_year
1,Mercedes,13,573.0,2020
2,Red Bull,2,319.0,2020
3,Racing Point,1,210.0,2020
4,McLaren,0,202.0,2020
5,Renault,0,181.0,2020
6,Ferrari,0,131.0,2020
7,AlphaTauri,1,107.0,2020
8,Alfa Romeo,0,8.0,2020
9,Haas F1 Team,0,3.0,2020
10,Williams,0,0.0,2020


In [0]:
# Lets write this into presentation layer
constructor_standings_final.write.mode("overwrite").parquet(f"{presentation_container_path}/constructor_standings")

In [0]:
dbutils.notebook.exit("success")